In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [ ]:
file = open(r'/content/drive/MyDrive/poetry (1).txt', encoding='utf-8')

In [ ]:
lines = file.readlines()

In [ ]:
wholepoem = ' '.join(lines)

In [ ]:
words = wholepoem.replace('\n', ' ').replace('\u200c', ' ').split()

In [ ]:
wordCount = {}
for word in tqdm(words):
    try:
        wordCount[word] += 1
    except:
        wordCount[word] = 1

100%|██████████| 14248/14248 [00:00<00:00, 1592380.38it/s]


In [ ]:
threshold = int(np.mean(np.array(list(wordCount.values()))) + 2*np.std(np.array(list(wordCount.values()))))

In [ ]:
filteredWords = []
for word in list(wordCount.keys()):
    if wordCount[word] > threshold:
        filteredWords.append(word)

In [ ]:
print(filteredWords)

['به', 'و', 'از', 'در', 'را', 'می', 'ز', 'بی', 'بر', 'همه', 'نه', 'که', 'هر', 'ما', 'تو', 'با', 'سعدی', 'آن', 'ای', 'چه', 'سر', 'یا', 'گر', 'دوست', 'جان', 'اگر', 'بود', 'تا', 'دست', 'چو', 'عشق', 'روی', 'است', 'چون', 'نمی', 'من', 'چشم', 'دل', 'این', 'ست']


In [ ]:
stopwords = ['به', 'و', 'از', 'در', 'را', 'می', 'ز', 'بی', 'بر', 'همه', 'نه', 'که', 'هر', 'ما', 'تو', 'با', 'آن', 'ای', 'چه', 'سر', 'یا', 'گر', 'اگر', 'بود', 'تا', 'چو', 'است', 'چون', 'نمی', 'من', 'این', 'ست']

In [ ]:
for stopword in stopwords:
    while True:
        try:
            words.remove(stopword)
        except:
            break

In [ ]:
dictionary = {}
indexer = 0
for word in words:
    try:
        dictionary[word]
    except:
        dictionary[word] = indexer
        indexer += 1

In [ ]:
data = pd.DataFrame(words, columns=['words_1'])

In [ ]:
data['words_2'] = data['words_1'].shift(1)
data['words_3'] = data['words_1'].shift(2)
data['words_4'] = data['words_1'].shift(3)
data['words_5'] = data['words_1'].shift(4)
data['target'] = data['words_1'].shift(-1)
data = data.dropna()

In [ ]:
def func(word):
    return dictionary[word]

In [ ]:
data['words_1'] = data['words_1'].apply(func)
data['words_2'] = data['words_2'].apply(func)
data['words_3'] = data['words_3'].apply(func)
data['words_4'] = data['words_4'].apply(func)
data['words_5'] = data['words_5'].apply(func)
data['target'] = data['target'].apply(func)

In [ ]:
X = data.drop('target', axis=1).values
y = data[['target']].values

In [ ]:
total_words = len(list(dictionary.keys()))

In [ ]:
y = to_categorical(y)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 20, input_length=5))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.3))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5, 20)             66600     
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               152576    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 3330)              855810    
Total params: 1,074,986
Trainable params: 1,074,986
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
class myCallback(keras.callbacks.Callback):
    def __init__(self):
        self.best = np.inf
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('loss') < self.best:
            model.save('saadi.hdf5')
            self.best = logs.get('loss')

In [ ]:
model.fit(X, y, batch_size=64, epochs=50, callbacks=[myCallback()])

Epoch 1/50
145/145 [==============================] - 6s 8ms/step - loss: 7.9811 - accuracy: 0.0095
Epoch 2/50
145/145 [==============================] - 1s 8ms/step - loss: 7.4502 - accuracy: 0.0091
Epoch 3/50
145/145 [==============================] - 1s 8ms/step - loss: 7.3379 - accuracy: 0.0134
Epoch 4/50
145/145 [==============================] - 1s 8ms/step - loss: 7.1522 - accuracy: 0.0105
Epoch 5/50
145/145 [==============================] - 1s 7ms/step - loss: 7.0492 - accuracy: 0.0104
Epoch 6/50
145/145 [==============================] - 1s 8ms/step - loss: 6.8701 - accuracy: 0.0137
Epoch 7/50
145/145 [==============================] - 1s 8ms/step - loss: 6.8050 - accuracy: 0.0138
Epoch 8/50
145/145 [==============================] - 1s 8ms/step - loss: 6.6924 - accuracy: 0.0127
Epoch 9/50
145/145 [==============================] - 1s 8ms/step - loss: 6.5684 - accuracy: 0.0157
Epoch 10/50
145/145 [==============================] - 1s 8ms/step - loss: 6.4599 - accuracy: 0.0152

In [ ]:
model.load_weights('saadi.hdf5')

In [ ]:
def produce(sentence, num_words):
    for _ in range(num_words):
        sentence = sentence.split()
        for stopword in stopwords:
            while True:
                try:
                    sentence.remove(stopword)
                except:
                    break
        sentence_cpy = sentence[-5:].copy()
        for i in range(len(sentence_cpy)):
            sentence_cpy[i] = dictionary[sentence_cpy[i]]
        if len(sentence_cpy) < 5:
           for i in range(5 - len(sentence_cpy)):
              sentence_cpy.insert(0 , 0)
        sentence.append(list(dictionary.keys())[np.argmax(model.predict(np.array(sentence_cpy).reshape(-1,5)))])
        sentence = ' '.join(sentence)
    return sentence

In [ ]:
sentence1 = 'قسمت خود می خورند منعم و درویش'
sentence2 = 'جانور از نطفه می کند شکر از نی'
sentence3 = 'خود نه زبان در دهان عارف مدهوش'
sentence4 = 'سعدی از آن جا که فهم اوست سخن گفت'
sentence5 = 'کآینه پاکیزه است و روی تو زیبا'
print(produce(sentence1, 5))
print(produce(sentence2, 5))
print(produce(sentence3, 5))
print(produce(sentence4, 5))
print(produce(sentence5, 5))

قسمت خود خورند منعم درویش نکوست ور یار پای موی
جانور نطفه کند شکر نی فراق کند ناب مقام علی
خود زبان دهان عارف مدهوش پیشت بدفرجام مگرت ماندم نظر
سعدی جا فهم اوست سخن گفت برد روی وصل رضا چشمه
کآینه پاکیزه روی زیبا نظیر طالع مگر کردیم خرقه
